<a href="https://colab.research.google.com/github/asvarella/MachineLearning-Trab1/blob/main/T1_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>